## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/DSL/2022-2/[0830] TextMining_WordEmbedding

/content/drive/MyDrive/DSL/2022-2/[0830] TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,099 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,076 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvid

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['‘내장형 심장감시장치 장착 테이저’',
 '테이저 때문에 죽는 사람은 의외로 많다',
 '‘워싱턴 포스트’ 지의 2015년 보도에 따르면, 경찰에게 테이저로 사격을 당한 후 구속 중 사망하는 사람이 1주에 1명 꼴로 발생한다고 한다',
 '전도성 전기 병기인 테이저로 인한 중상자 대부분은 근육 마비에 의한 낙상으로 인해 생긴다',
 '그러나 증거에 따르면 테이저가 심장 마비를 일으킬 수 있는 것으로 나타났다',
 '현재 여러 연구자들은 테이저에 EKG(심전도) 기기를 추가하면 피의자의 심부담을 측정함으로서 피의자의 심장 위험을 막을 수 있다고 생각한다',
 '웨이크 포리스트 뱁티스트 메디컬 센터의 응급 의학교수인 윌리엄 보즈먼은 지난 2009년 동료들과 함께, 테이저의 안전성을 철저히 검증했다',
 '그들은 테이저에 명중된 사람은 99.75%가 아무 부상이 없거나 경상 이하만을 입는다고 결론지었다',
 '보즈먼 교수는 “테이저는 놀랄 만큼 효율적이고 또한 안전합니다',
 '경찰관들이 휴대하는 총기나 가스 분사기, 경봉, 맨손 제압 등 다른 체포 수단과 비교해 볼 때 그 어떤 것보다도 안전합니다',
 '그러나 세상에 100%는 없습니다',
 '구류 중 돌연사가 꾸준히 일어나는 게 그 증거입니다.”라고 말한다',
 '테이저에 피격된 사람 250만 명 중 1명은 심장에 무리가 가고, 이 중 일부는 결국 그로 인해 죽게 된다',
 '“테이저에 피격된 사람이 의학적으로 문제가 있음을 알게 된다면, 신속하게 응급조치를 취해 그 사람의 생명을 구할 수 있습니다.”',
 '1년에 테이저 때문에 죽는 사람이 정확히 몇 명인지는 확실히 발표된 바가 없지만, 보즈먼과 그의 동료들은 더욱 안전한 테이저를 만들고자 한다',
 '보즈먼은 테이저의 기본 구조가 EKG 기기와 의외로 유사하다는 데 착안했다',
 '두 개의 탐침이 전선을 통해 전자 기기와 연결되어 있는 것이다',
 '보즈먼과 동료인 제이슨 스토피라 박사는 두 기기를 결합하는 작업이 가능할 거라고 생각했다.이들은 미국 사

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [10]:
okt = Okt()

In [30]:
import pandas as pd 

In [72]:
stop_words_list= pd.read_csv('/content/drive/MyDrive/DSL/2022-2/[0830] TextMining_WordEmbedding/stop.txt',names= ['column'],header=None)

In [75]:
stop_words_list = stop_words_list['column'].tolist()

In [99]:
stop_words_list

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

https://www.ranks.nl/stopwords/korean => korean stopwords 를 txt로 저장하고 리스트로 변환하여 제거하는데 이용하였습니다.

In [96]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence, stop = ['Josa','Suffix','Punctuation', 'Foreign', 'Number']):
  #stop_words는 korean stopwords입니다. 
  stop_words = set(stop_words_list)
  word_tokens=[]
  for i, j in okt.pos(sentence):
    if j in stop:
      continue
    word_tokens.append(i)
  
  word_tokens = [word for word in word_tokens if not word in stop_words]

  return word_tokens

In [97]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [98]:
corpus_tokenized[0]

['내장', '심장', '감시', '장치', '장착', '테이저']

In [ ]:
corpus_tokenized

# 2번


이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [104]:
len(corpus_tokenized)

1020

In [146]:
len(corpus_tokenized[1])

6

In [109]:
len(corpus_tokenized[2])

21

---



사이즈는 어떻게 설정 해야 할지 몰라서 default인 100으로 그대로 설정하였고, 

 vocabulary 의 사이즈를 보니 1020개의 문장에 각각 평균 10개정도의 토큰이 있는 것 같아 . 왜냐하면 많은 부분을 stop words로 지정하여 전처리하기도 했고, 조사, punctuation 부분들도 모두 제거하였기 때문입니다. 


그리고 세션때 Cbow 보다 skipgram이 성능이 우수하였다고 말씀하셨기에 sg=1로 설정하였습니다. 





In [101]:
from gensim.models import Word2Vec

In [165]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=100, window=5, min_count=1, workers=4, sg=1,alpha=0.025)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 4437


In [166]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20 )

(291404, 307940)

In [167]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용
# 모델 학습
model_no_pretrained.wv.vocab.keys()

dict_keys(['내장', '심장', '감시', '장치', '장착', '테이저', '때문', '죽는', '사람', '의외로', '많다', '워싱턴', '포스트', '지의', '보도', '따르면', '경찰', '사격', '당한', '후', '구속', '중', '사망', '하는', '주', '명', '꼴', '발생', '한다고', '한다', '전도성', '전기', '병기', '인한', '중상', '대부분', '근육', '마비', '의한', '낙상', '인해', '생긴다', '증거', '일으킬', '수', '있는', '나타났다', '현재', '여러', '연구자', 'EKG', '심전도', '기기', '추가', '하면', '피의자', '심', '부담', '측정', '함', '위험', '막', '있다고', '생각', '웨이', '크', '포리스트', '뱁티스트', '메디', '컬', '센터', '응급', '의학', '교수', '윌리엄', '보즈', '먼은', '지난', '동료', '안전성', '철저히', '검증', '했다', '명중', '된', '아무', '부상', '없거나', '경상', '이하', '입는다고', '결론', '지었다', '먼', '놀랄', '만큼', '효율', '안전합니다', '경찰관', '휴대', '총기', '가스', '분사기', '경봉', '맨손', '제압', '체포', '수단', '비교', '해', '볼', '세상', '는', '없습니다', '구류', '돌연사', '꾸준히', '일어나는', '게', '입니다', '말', '피격', '무리', '가고', '일부', '그로', '죽게', '된다', '문제', '있', '음', '알', '된다면', '신속하게', '조치', '취해', '생명', '할', '있습니다', '정확히', '명인', '확실히', '발표', '바', '없지만', '먼과', '더욱', '안전한', '만들고자', '기본', '구조', '유사하다는', '데', '착안', '두', '개', '탐', '침', '전선', '통해', '연결

In [168]:
model_no_pretrained.wv.most_similar('암')

[('나왔다', 0.9900789856910706),
 ('한다는', 0.9891387820243835),
 ('치료', 0.9867357015609741),
 ('사이언스', 0.9867210984230042),
 ('중인', 0.9860175848007202),
 ('Nature', 0.9856899976730347),
 ('어플', 0.9854773283004761),
 ('선천성', 0.9846386313438416),
 ('커뮤니케이션', 0.9842337965965271),
 ('온라인', 0.9839595556259155)]

In [169]:
import gensim

In [172]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, window=5, min_count=1, workers=4, sg=1,alpha=0.025)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 4437


In [171]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 2002


In [173]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 40 )

(582607, 615880)

In [174]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('암')

[('당뇨병', 0.8946239948272705),
 ('아스피린', 0.8903173208236694),
 ('장기', 0.8775237798690796),
 ('선천성', 0.8733351826667786),
 ('복용', 0.8672981262207031),
 ('혈우병', 0.8581836819648743),
 ('기형', 0.8506456613540649),
 ('입원', 0.8464334011077881),
 ('전이', 0.8435022830963135),
 ('폐암', 0.8293088674545288)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

In [175]:
model_no_pretrained.wv.most_similar('암')

[('나왔다', 0.9900789856910706),
 ('한다는', 0.9891387820243835),
 ('치료', 0.9867357015609741),
 ('사이언스', 0.9867210984230042),
 ('중인', 0.9860175848007202),
 ('Nature', 0.9856899976730347),
 ('어플', 0.9854773283004761),
 ('선천성', 0.9846386313438416),
 ('커뮤니케이션', 0.9842337965965271),
 ('온라인', 0.9839595556259155)]

In [176]:
model_pretrained.wv.most_similar('암')

[('당뇨병', 0.8946239948272705),
 ('아스피린', 0.8903173208236694),
 ('장기', 0.8775237798690796),
 ('선천성', 0.8733351826667786),
 ('복용', 0.8672981262207031),
 ('혈우병', 0.8581836819648743),
 ('기형', 0.8506456613540649),
 ('입원', 0.8464334011077881),
 ('전이', 0.8435022830963135),
 ('폐암', 0.8293088674545288)]


먼저 사전학습 없이 돌린 모델도 결과에 서로 선천성과,뇌종양과 같은 키워드가 서로 공통적으로 나왔습니다. 하지만 pretrain을 하지 않은 모델의 경우 유사도가 corpus데이터에 다소 오버피팅된 경향을 보였습니다. 

특히 한다는, 나왔다 등의 일상적인 단어가 높은 순위에 기록된 반면 pretrained를 거친 모델은 이런 일상적인 단어는 나오지 않았음을 확인할 수 있습니다. 이는 데이터 셋의 부족함과 또 주변의 몇개로 학습을 진행할지 결정하는 window의 하이퍼파라미터가 조금 조절되어야 할 것 같다라는 생각을 하게됐습니다! 

또 처음 모델을 생성할때는 lr을 0.01로 했더니 너무 학습이 이상하게 돼어 다시 기본값인 0.25로 수정했더니 좀 더 나은 결과가 나왔습니다. 데이터셋의 상황에 맞게 alpha (Lr값의 설정을 어떻게 해야할지 고민이 들었습니다. 

---


# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [177]:
x=list(set(sum(corpus_tokenized,[])))
print(x)

['어려워', '극', '익숙해진', '작았던', '참석', '정답', '했던', '걸쳐서', '선보일', '돌기', 'IBS', '귀중한', '만들', '초점', '특징', '나타내는', '떨어집니다', '메디', '재진', '통화', 'GPU', '어렵거나', '한꺼', '있었던', '기준', '쉽게', '보내', '덧붙이는', '엠블럼', 'Tolgoi', '산업혁명', '붉게', '외날찍', 'CT', '찾아', '표지', '유인', '오늘날', '나타내지', '급사', '끼쳐', '학연', '확장', '연구', '광고', '슬랙', '바이', '발전', '차량', '독한', '높은', '사법', 'Sido', '공정', '네이처', 'Network', '직업', '시켜야', '만드는', 'deite', '인적자원', '상', 'specific', '주인', '양자역학', '부상', '비밀', '메타', '구단', '했기', '서는', '되도록', '세계보건기구', '지원', '육성', '비율', '판단', '려고', '바람', '첨단', '출신', '본', '석', '바탕', '남아', '연설', '내리는지에', '만나기', '취해', '침', '치명', '성했다', '알려진', '맺', '같아서', '화학반응', '임박', '되는', '했느냐', '노선', '시연', '되고', '뱅크', '맡겼다', '다공', '현저', '녹이면', '로이스', '상당수', '자폐', '당시', '시정', '상용', '처리장치', '한다', '집속', '약물', '나타난다', '나라', 'airway', '리', '산맥', '위원회', '열린', '펼치고', '근본', '마리', '조립', '추세', '가닥', '탐사', '칸국', 'GRP', '격', '대사', '졸게까지', '접거나', '생길', '급박하게', '현대인', '유선', '실업', '그룹', '여성', '미래창조과학부', '홍수', '무질서', '즉각', '로터스', '차오른', '잘라', '런던'

In [178]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

4437

In [ ]:
unique_text

In [191]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_no_pretrained.wv[i])

In [181]:
import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [192]:
pca = PCA(random_state = 0).fit_transform(vector)
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.05, init = 'random', n_iter = 1000).fit_transform(vector)

In [187]:
X, t = make_swiss_roll(n_samples = 1000, noise = 0)

In [188]:
color = (t - t.min()) / (t.max() - t.min())

In [196]:
tsne.shape

(4437, 2)

In [197]:
tsne

array([[-0.01816879,  0.254663  ],
       [ 0.08317935, -0.30664426],
       [-0.03794162,  0.1948504 ],
       ...,
       [ 0.13335025, -0.40459967],
       [-0.08696826,  0.26122442],
       [ 0.272671  , -0.46172485]], dtype=float32)

In [193]:
# 시각화해보세요!

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(tsne.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [194]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(pca.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [199]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 20, learning_rate = 0.05, init = 'random', n_iter = 2000).fit_transform(vector)
# 시각화해보세요!

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(tsne.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!